In [ ]:
# Importovanje biblioteka
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Podešavanje vizuelizacija
plt.style.use('default')
sns.set_palette('husl')
%matplotlib inline

## 1.1 Učitavanje skupa podataka

In [ ]:
# Učitavanje podataka
df = pd.read_csv('../data/dataset.csv')

print(f"Skup podataka učitan uspešno!")
print(f"Dimenzije: {df.shape[0]} redova, {df.shape[1]} kolona")

In [ ]:
# Pregled prvih nekoliko redova
print("\nPrvih 5 redova:")
df.head()

In [ ]:
# Informacije o kolonama i tipovima podataka
print("\nInformacije o podacima:")
df.info()

## 1.2 Provera nedostajućih vrednosti

In [ ]:
# Provera nedostajućih vrednosti
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    'Kolona': missing_values.index,
    'Broj nedostajućih': missing_values.values,
    'Procenat': missing_percentage.values
})

missing_df = missing_df[missing_df['Broj nedostajućih'] > 0].sort_values('Broj nedostajućih', ascending=False)

if len(missing_df) > 0:
    print("\nNedostajuće vrednosti:")
    print(missing_df.to_string(index=False))
else:
    print("\n✓ Nema nedostajućih vrednosti u skupu podataka!")

## 1.3 Osnovna analiza podataka

In [ ]:
# Statistika za numeričke kolone
print("\nStatistika numeričkih obeležja:")
df.describe()

In [ ]:
# Identifikacija numeričkih i kategoričkih kolona
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"\nNumeričke kolone ({len(numeric_cols)}):")
print(numeric_cols)
print(f"\nKategoričke kolone ({len(categorical_cols)}):")
print(categorical_cols)

## 1.4 Analiza ciljnog obeležja (popularnost)

In [ ]:
# Statistika popularity kolone
if 'popularity' in df.columns:
    print("\nStatistika popularnosti:")
    print(f"Min: {df['popularity'].min()}")
    print(f"Max: {df['popularity'].max()}")
    print(f"Srednja vrednost: {df['popularity'].mean():.2f}")
    print(f"Medijana: {df['popularity'].median():.2f}")
    print(f"Standardna devijacija: {df['popularity'].std():.2f}")
else:
    print("\nKolona 'popularity' nije pronađena. Proverite ime kolone.")

In [ ]:
# Distribucija popularnosti
if 'popularity' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Histogram
    axes[0].hist(df['popularity'], bins=50, edgecolor='black', alpha=0.7)
    axes[0].set_xlabel('Popularnost', fontsize=12)
    axes[0].set_ylabel('Frekvencija', fontsize=12)
    axes[0].set_title('Distribucija popularnosti pesama', fontsize=14, fontweight='bold')
    axes[0].axvline(df['popularity'].mean(), color='red', linestyle='--', label=f'Srednja vrednost: {df["popularity"].mean():.2f}')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Box plot
    axes[1].boxplot(df['popularity'], vert=True)
    axes[1].set_ylabel('Popularnost', fontsize=12)
    axes[1].set_title('Box Plot popularnosti', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Analiza distribucije po opsezima popularnosti
if 'popularity' in df.columns:
    bins = [0, 20, 40, 60, 80, 100]
    labels = ['0-20', '21-40', '41-60', '61-80', '81-100']
    df['popularity_range'] = pd.cut(df['popularity'], bins=bins, labels=labels, include_lowest=True)
    
    popularity_dist = df['popularity_range'].value_counts().sort_index()
    
    print("\nDistribucija po opsezima popularnosti:")
    for range_name, count in popularity_dist.items():
        percentage = (count / len(df)) * 100
        print(f"{range_name}: {count} pesama ({percentage:.2f}%)")
    
    # Vizuelizacija
    plt.figure(figsize=(10, 6))
    bars = plt.bar(popularity_dist.index, popularity_dist.values, edgecolor='black', alpha=0.7)
    plt.xlabel('Opseg popularnosti', fontsize=12)
    plt.ylabel('Broj pesama', fontsize=12)
    plt.title('Distribucija pesama po opsezima popularnosti', fontsize=14, fontweight='bold')
    plt.xticks(rotation=0)
    
    # Dodavanje vrednosti na barove
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}\n({height/len(df)*100:.1f}%)',
                ha='center', va='bottom', fontsize=10)
    
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
    
    # Uklanjamo privremenu kolonu
    df.drop('popularity_range', axis=1, inplace=True)

In [ ]:
# Kreiranje binarne klasifikacije (popularno/nepopularno)
if 'popularity' in df.columns:
    threshold = 50
    df['is_popular'] = (df['popularity'] > threshold).astype(int)
    
    print(f"\nBinarna klasifikacija (prag: {threshold}):")
    class_dist = df['is_popular'].value_counts()
    print(f"Nepopularne pesme (0): {class_dist[0]} ({class_dist[0]/len(df)*100:.2f}%)")
    print(f"Popularne pesme (1): {class_dist[1]} ({class_dist[1]/len(df)*100:.2f}%)")
    print(f"\nOdnos klasa: 1:{class_dist[0]/class_dist[1]:.2f}")
    
    # Vizuelizacija
    plt.figure(figsize=(8, 6))
    bars = plt.bar(['Nepopularne (≤50)', 'Popularne (>50)'], class_dist.values, 
                   color=['#ff7f0e', '#2ca02c'], edgecolor='black', alpha=0.7)
    plt.ylabel('Broj pesama', fontsize=12)
    plt.title('Distribucija klasa (Popularne vs Nepopularne)', fontsize=14, fontweight='bold')
    
    # Dodavanje vrednosti na barove
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}\n({height/len(df)*100:.1f}%)',
                ha='center', va='bottom', fontsize=11)
    
    plt.grid(True, alpha=0.3, axis='y')
    plt.tight_layout()
    plt.show()
    
    # Uklanjamo privremenu kolonu
    df.drop('is_popular', axis=1, inplace=True)

## 1.5 Sačuvaj izvještaj

Sačuvaćemo osnovne informacije o skupu podataka za dalju referencu.

In [ ]:
# Kreiranje izveštaja
report = {
    'Broj redova': df.shape[0],
    'Broj kolona': df.shape[1],
    'Nedostajuće vrednosti': df.isnull().sum().sum(),
    'Duplikati': df.duplicated().sum()
}

print("\n" + "="*50)
print("IZVEŠTAJ O SKUPU PODATAKA")
print("="*50)
for key, value in report.items():
    print(f"{key}: {value}")
print("="*50)

## Zaključak Celine 1

U ovoj celini smo:
- ✓ Uspešno učitali skup podataka
- ✓ Proverili nedostajuće vrednosti
- ✓ Izvršili osnovnu analizu podataka
- ✓ Analizirali distribuciju ciljnog obeležja (popularnost)
- ✓ Utvrdili neuravnoteženost klasa u podacima

**Sledeća celina:** Pretprocesiranje podataka (obrada nedostajućih vrednosti, transformacija kategoričkih promenljivih, kreiranje ciljne promenljive, balansiranje klasa)